In [1]:
import warnings
warnings.filterwarnings(action='ignore')

from pathlib import Path

import imageio
import numpy as np
from IPython.display import Image
import torch
import transformers
from transformers import TrainingArguments, Trainer, AutoProcessor, AutoModel, VideoMAEImageProcessor, VideoMAEForVideoClassification
import pandas as pd
import pytorchvideo
import pytorchvideo.data
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    RemoveKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import evaluate

2023-03-13 03:05:02.141792: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 03:05:02.648893: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64/
2023-03-13 03:05:02.648940: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64/
2023-03-13 03:05:02.648945: W tensorflow/compiler/tf2tensorrt/utils/py_u

In [2]:
# def get_df(method):
#     train_df = pd.read_csv(Path(dataset_root_path, "train.csv"))
    
#     if method == "crash":
#         train_df.loc[train_df["label"]!=0, "label"] = 1
    
#     elif method == "ego_involve":
#         train_df.drop(train_df[train_df["label"]==0].index, inplace=True)
#         train_df.loc[(train_df["label"]==1)|(train_df["label"]==2)|(train_df["label"]==3)|(train_df["label"]==4)|(train_df["label"]==5)|(train_df["label"]==6), "label"] = 0 # yes
#         train_df.loc[(train_df["label"]==7)|(train_df["label"]==8)|(train_df["label"]==9)|(train_df["label"]==10)|(train_df["label"]==11)|(train_df["label"]==12), "label"] = 1 # no
        
#     elif method == "weather":
#         train_df.drop(train_df[(train_df["label"]==0)].index, inplace=True)
#         train_df.loc[(train_df["label"]==1)|(train_df["label"]==2)|(train_df["label"]==7)|(train_df["label"]==8), "label"] = 0 # normal
#         train_df.loc[(train_df["label"]==3)|(train_df["label"]==4)|(train_df["label"]==9)|(train_df["label"]==10), "label"] = 1 # snowy
#         train_df.loc[(train_df["label"]==5)|(train_df["label"]==6)|(train_df["label"]==11)|(train_df["label"]==12), "label"] = 2 # rainy
        
#     elif method == "timing":
#         train_df.drop(train_df[train_df["label"]==0].index, inplace=True)
#         train_df.loc[(train_df["label"]==1)|(train_df["label"]==3)|(train_df["label"]==5)|(train_df["label"]==7)|(train_df["label"]==9)|(train_df["label"]==11), "label"] = 0 # day
#         train_df.loc[(train_df["label"]==2)|(train_df["label"]==4)|(train_df["label"]==6)|(train_df["label"]==8)|(train_df["label"]==10)|(train_df["label"]==12), "label"] = 1 # night
    
#     train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])
    
#     return train_df, val_df

# dataset_root_path = "/home/djlee/deep/datasets/open"

# for method in ["crash", "ego_involve", "weather", "timing"]:
#     train_df, val_df = get_df(method=method)

#     for phase, df in [("train", train_df), ("val", val_df)]:
#         with open(Path(dataset_root_path, f"{method}_{phase}.txt"), "w") as f:
#             for filename, label in df[["sample_id", "label"]].values:
#                 filename = f"{filename}.mp4"
#                 f.write(f"{filename} {label}\n")

In [3]:
# def unnormalize_img(img):
#     """Un-normalizes the image pixels."""
#     img = (img * std) + mean
#     img = (img * 255).astype("uint8")
#     return img.clip(0, 255)

# def create_gif(video_tensor, filename="sample.gif"):
#     """Prepares a GIF from a video tensor.
    
#     The video tensor is expected to have the following shape:
#     (num_frames, num_channels, height, width).
#     """
#     frames = []
#     for video_frame in video_tensor:
#         frame_unnormalized = unnormalize_img(video_frame.permute(1, 2, 0).numpy())
#         frames.append(frame_unnormalized)
#     kargs = {"duration": 0.25}
#     imageio.mimsave(filename, frames, "GIF", **kargs)
#     return filename

# def display_gif(video_tensor, gif_name="sample.gif"):
#     """Prepares and displays a GIF from a video tensor."""
#     video_tensor = video_tensor.permute(1, 0, 2, 3)
#     print(video_tensor.shape)
#     gif_filename = create_gif(video_tensor, gif_name)
#     return Image(filename=gif_filename)

# sample_video = next(iter(train_dataset))
# video_tensor = sample_video["video"]
# display_gif(video_tensor)

In [4]:
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=0.25, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = torch.nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1-pt)**self.gamma * ce_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [5]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        criterion = FocalLoss()
        loss = criterion(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [6]:
def run(method):
    if method == "crash":
        class_labels = ["no", "yes"]
    elif method == "ego_involve":
        class_labels = ["yes", "no"]
    elif method == "weather":
        class_labels = ["normal", "snow", "rainy"]
    elif method == "timing":
        class_labels = ["day", "night"]
    
    label2id = {label: i for i, label in enumerate(class_labels)}
    id2label = {i: label for label, i in label2id.items()}

    model_ckpt = "MCG-NJU/videomae-base-finetuned-kinetics"
    image_processor = VideoMAEImageProcessor.from_pretrained(model_ckpt)
    model = VideoMAEForVideoClassification.from_pretrained(
        model_ckpt,
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,  # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
    )

    dataset_root_path = "/home/djlee/deep/datasets/open"
    mean = image_processor.image_mean
    std = image_processor.image_std
    if "shortest_edge" in image_processor.size:
        height = width = image_processor.size["shortest_edge"]
    else:
        height = image_processor.size["height"]
        width = image_processor.size["width"]
    resize_to = (height, width)

    num_frames_to_sample = model.config.num_frames
    sample_rate = 1
    fps = 10
    clip_duration = 5

    # Training dataset transformations.
    train_transform = Compose(
        [
            ApplyTransformToKey(
                key="video",
                transform=Compose(
                    [
                        UniformTemporalSubsample(num_frames_to_sample),
                        Lambda(lambda x: x / 255.0),
                        Normalize(mean, std),
                        ShortSideScale(256),
                        RandomCrop(resize_to),
                        RandomHorizontalFlip(p=0.5),
                    ]
                ),
            ),
        ]
    )

    # Validation and evaluation datasets' transformations.
    val_transform = Compose(
        [
            ApplyTransformToKey(
                key="video",
                transform=Compose(
                    [
                        UniformTemporalSubsample(num_frames_to_sample),
                        Lambda(lambda x: x / 255.0),
                        Normalize(mean, std),
                        Resize(resize_to),
                    ]
                ),
            ),
        ]
    )

    # Training dataset.
    train_dataset = pytorchvideo.data.Ucf101(
        data_path=Path(dataset_root_path, f"{method}_train.txt"),
        video_path_prefix=Path(dataset_root_path, "train"),
        clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
        decode_audio=False,
        transform=train_transform,
    )

    # Validation and evaluation datasets.
    val_dataset = pytorchvideo.data.Ucf101(
        data_path=Path(dataset_root_path, f"{method}_val.txt"),
        video_path_prefix=Path(dataset_root_path, "train"),
        clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
        decode_audio=False,
        transform=val_transform,
    )
    test_dataset = pytorchvideo.data.Ucf101(
        data_path=Path(dataset_root_path, "test_label.txt"),
        video_path_prefix=Path(dataset_root_path, "test"),
        clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
        decode_audio=False,
        transform=val_transform,
    )
    print(train_dataset.num_videos, val_dataset.num_videos, test_dataset.num_videos)

    model_name = model_ckpt.split("/")[-1]
    new_model_name = f"{model_name}-{method}"
    num_epochs = 10
    batch_size = 4
    gradient_accumulation_steps = 1

    args = TrainingArguments(
        new_model_name,
        remove_unused_columns=False,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_ratio=0.1,
        logging_steps=10,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        dataloader_num_workers=4,
        gradient_checkpointing=True,
        gradient_accumulation_steps=gradient_accumulation_steps,
        fp16=True,
        do_train=True,
        do_eval=True,
        max_steps=(train_dataset.num_videos // batch_size) * num_epochs,
    )

    metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        predictions = np.argmax(eval_pred.predictions, axis=1)
        result = metric.compute(predictions=predictions, references=eval_pred.label_ids)
        result["f1"] = f1_score(eval_pred.label_ids, predictions, average='macro')
        return result

    def collate_fn(examples):
        # permute to (num_frames, num_channels, height, width)
        pixel_values = torch.stack(
            [example["video"].permute(1, 0, 2, 3) for example in examples]
        )
        labels = torch.tensor([example["label"] for example in examples])
        return {"pixel_values": pixel_values, "labels": labels}

    trainer = CustomTrainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=image_processor,
        compute_metrics=compute_metrics,
        data_collator=collate_fn,
    )

    train_results = trainer.train()
    predict = trainer.predict(test_dataset)
    return train_results, predict

In [7]:
result = {}
for method in ["crash", "ego_involve", "weather", "timing"]:
    result[method] = {}
    result[method]["r"], result[method]["p"] = run(method=method)

Some weights of VideoMAEForVideoClassification were not initialized from the model checkpoint at MCG-NJU/videomae-base-finetuned-kinetics and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([400, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([400]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2158 540 1800


max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
***** Running training *****
  Num examples = 21560
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 5390
  Number of trainable parameters = 86228738


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.003400,0.008353,0.959259,0.953078
1,0.028800,0.002445,0.988889,0.987602
2,0.002000,0.001113,0.994444,0.993776
3,0.000200,0.001152,0.994444,0.993825
4,0.000100,0.000133,1.000000,1.000000
5,0.000000,0.000005,1.000000,1.000000
6,0.000000,0.000001,1.000000,1.000000
7,0.000000,0.000376,0.998148,0.997936
8,0.000000,0.000001,1.000000,1.000000
9,0.000000,0.000001,1.000000,1.000000


***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 4
Saving model checkpoint to videomae-base-finetuned-kinetics-crash/checkpoint-540
Configuration saved in videomae-base-finetuned-kinetics-crash/checkpoint-540/config.json
Model weights saved in videomae-base-finetuned-kinetics-crash/checkpoint-540/pytorch_model.bin
Image processor saved in videomae-base-finetuned-kinetics-crash/checkpoint-540/preprocessor_config.json
***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 4
Saving model checkpoint to videomae-base-finetuned-kinetics-crash/checkpoint-1080
Configuration saved in videomae-base-finetuned-kinetics-crash/checkpoint-1080/config.json
Model weights saved in videomae-base-finetuned-kinetics-crash/checkpoint-1080/pytorch_model.bin
Image processor saved in videomae-base-finetuned-kinetics-crash/checkpoint-1080/preprocessor_config.json
***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 4
Saving model checkpoint to videomae-base-f

...

732 183 1800


max_steps is given, it will override any value given in num_train_epochs
Using cuda_amp half precision backend
***** Running training *****
  Num examples = 7320
  Num Epochs = 9223372036854775807
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1830
  Number of trainable parameters = 86229507


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.060600,0.034085,0.857923,0.628152
1,0.063300,0.044186,0.754098,0.690087


***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 4
Saving model checkpoint to videomae-base-finetuned-kinetics-weather/checkpoint-184
Configuration saved in videomae-base-finetuned-kinetics-weather/checkpoint-184/config.json
Model weights saved in videomae-base-finetuned-kinetics-weather/checkpoint-184/pytorch_model.bin
Image processor saved in videomae-base-finetuned-kinetics-weather/checkpoint-184/preprocessor_config.json
***** Running Evaluation *****
  Num examples: Unknown
  Batch size = 4
Saving model checkpoint to videomae-base-finetuned-kinetics-weather/checkpoint-368
Configuration saved in videomae-base-finetuned-kinetics-weather/checkpoint-368/config.json
Model weights saved in videomae-base-finetuned-kinetics-weather/checkpoint-368/pytorch_model.bin
Image processor saved in videomae-base-finetuned-kinetics-weather/checkpoint-368/preprocessor_config.json


RuntimeError: [enforce fail at inline_container.cc:337] . unexpected pos 42506240 vs 42506128

In [ ]:
z

In [ ]:
test_dataset = pytorchvideo.data.Ucf101(
    data_path=Path(dataset_root_path, "test_label.txt"),
    video_path_prefix=Path(dataset_root_path, "test"),
    clip_sampler=pytorchvideo.data.make_clip_sampler("uniform", clip_duration),
    decode_audio=False,
    transform=val_transform,
)